In [1]:
import json
import re
import sys
from math import factorial

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from datasets import Dataset
from eval_utils import (
    Format,
    PrintCallback,
    get_metrics_computer,
    get_to_string_processor,
)
from torch import nn
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
)

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-05-15 09:01:03.436716: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
def get_preprocessor(tokenizer, format):
    def preprocess_function(examples):
        inputs = [re.sub("\n", " <NL> ", text) for text in examples["Text"]]
        targets = [
            get_to_string_processor(format)(bundles) for bundles in examples["bundles"]
        ]
        model_inputs = tokenizer(
            inputs, text_target=targets, max_length=256, truncation=True
        )
        return model_inputs

    return preprocess_function

In [3]:
format = Format.SpecTokens

device = "cuda:0" if torch.cuda.is_available() else "cpu"
print(device)

cuda:0


In [4]:
train_data = pd.read_csv("~/work/resources/data/ads_train.csv")
train_data = train_data[train_data["n_bundles"] <= 5]
train_data.set_index(np.arange(len(train_data)), inplace=True)
print(f"We have train datset of size {len(train_data)}")
train_data.head()

We have train datset of size 8632


,Text,bundles,n_bundles
0,Продам телефон POCO F4 128GB телефону 8 месяце...,"[{""Title"": ""\u0442\u0435\u043b\u0435\u0444\u04...",1
1,"продам sup борд 10'6 - 320х76х15 см, пользова...","[{""Title"": ""sup \u0431\u043e\u0440\u0434 \u043...",1
2,Изменились планы . Отдам за 2000,[],0
3,Продам рацию MegaJet MJ-333 в хорошем рабочем ...,"[{""Title"": ""\u0440\u0430\u0434\u0438\u043e\u04...",1
4,Продам PlayStation 3\n3 джойстик и все диски\n...,"[{""Title"": ""PlayStation 3 \u0441 3 \u0434\u043...",1


In [5]:
eval_data = pd.read_csv("~/work/resources/data/ads_eval.csv")
eval_data = eval_data[:200]
eval_data.set_index(np.arange(len(eval_data)), inplace=True)
print(f"We have eval datset of size {len(eval_data)}")
eval_data.head()

We have eval datset of size 200


,Text,bundles,n_bundles
0,Распродажа \nПлатья \nЦена: 300р\nРазмер: 44-4...,"[{""Title"": ""\u041f\u043b\u0430\u0442\u044c\u04...",1
1,"вегетарианская энциклопедия вкусов, что с чем ...","[{""Title"": ""\u0432\u0435\u0433\u0435\u0442\u04...",1
2,"продам щетку для сухого массажа, новая (так и ...","[{""Title"": ""\u0429\u0435\u0442\u043a\u0430 \u0...",1
3,"Зимний тёплый комбинезон, пух, Crockid, p.74...","[{""Title"": ""\u0417\u0438\u043c\u043d\u0438\u04...",1
4,"удлинитель икеа 1,5 м на 6 розеток\n25 лари\n📍...","[{""Title"": ""\u0443\u0434\u043b\u0438\u043d\u04...",1


In [6]:
model_checkpoint = "ai-forever/ruT5-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer.add_tokens(["₾", "$", "€", "<NL>"], special_tokens=False)
if format == Format.SpecTokens:
    tokenizer.add_tokens(
        [
            "<BOB>",
            "<EOB>",
            "<BOT>",
            "<EOT>",
            "<BOP>",
            "<EOP>",
            "<BOC1>",
            "<EOC1>",
            "<BOC2>",
            "<EOC2>",
        ],
        special_tokens=False,
    )
elif format == Format.JustJson:
    tokenizer.add_tokens(["{", "}"], special_tokens=False)
model.resize_token_embeddings(len(tokenizer))

/home/jupyter/.local/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in th

Embedding(32113, 768)

In [7]:
train_dataset = Dataset.from_pandas(train_data[["Text", "bundles"]])
train_ads = train_dataset.map(
    get_preprocessor(tokenizer, format),
    batched=True,
    num_proc=4,
    remove_columns=train_dataset.column_names,
)
train_ads = train_ads.flatten()

eval_dataset = Dataset.from_pandas(eval_data[["Text", "bundles"]])
eval_ads = eval_dataset.map(
    get_preprocessor(tokenizer, format),
    batched=True,
    num_proc=4,
    remove_columns=eval_dataset.column_names,
)
eval_ads = eval_ads.flatten()

Parameter 'function'=<function get_preprocessor.<locals>.preprocess_function at 0x7fdbbabf5750> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.
Map (num_proc=4): 100%|██████████| 200/200 [00:00<00:00, 444.43 examples/s]


In [8]:
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer)

In [9]:
n_epochs = 10

training_args = Seq2SeqTrainingArguments(
    output_dir="ruT5",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=5e-5,  # 3e-5
    per_device_train_batch_size=8,  # 8
    per_device_eval_batch_size=8,  # 8
    weight_decay=0.5,  # 0.01
    save_total_limit=1,
    num_train_epochs=n_epochs,
    predict_with_generate=True,
    generation_max_length=256,
    lr_scheduler_type="cosine",
    group_by_length=False,
    warmup_steps=200,  # 200
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_ads,
    eval_dataset=eval_ads,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=get_metrics_computer(tokenizer, format),
    callbacks=[PrintCallback(format, show=5, device=device)],
)

In [10]:
trainer.train()

  5%|▍         | 500/10790 [01:48<33:25,  5.13it/s]

{'loss': 2.0693, 'grad_norm': 6.453489303588867, 'learning_rate': 4.9901059596372036e-05, 'epoch': 0.46}


  9%|▉         | 1000/10790 [04:28<35:20,  4.62it/s]  

{'loss': 0.4481, 'grad_norm': 2.4874117374420166, 'learning_rate': 4.9299257516933094e-05, 'epoch': 0.93}


100%|██████████| 25/25 [01:20<00:00,  3.88s/it]



                                                    
100%|██████████| 25/25 [05:31<00:00,  3.88s/it]

...

 51%|█████     | 5500/10790 [56:01<17:14,  5.11it/s]    

{'loss': 0.1743, 'grad_norm': 1.0153436660766602, 'learning_rate': 2.4962917947337045e-05, 'epoch': 5.1}


 56%|█████▌    | 6000/10790 [58:41<19:39,  4.06it/s]   

{'loss': 0.1508, 'grad_norm': 1.8940749168395996, 'learning_rate': 2.126870515925464e-05, 'epoch': 5.56}


KeyboardInterrupt: 

In [ ]:
output_dir = "good_checkpoints/ruT5-base"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)